In [1]:
import numpy as np
import speech_recognition as sr
import pyttsx3 as pt

In [4]:
# The speech to text module, which converts actively converts speech to text. Needs PyAudio to be installed for 
# using microphone input


# import speech_recognition as sr
# r = sr.Recognizer()
# with sr.Microphone() as source:
#     print("Speak Anything :")
#     audio = r.listen(source)
#     try:
#         text = r.recognize_google(audio) 
#         print("You said : {}".format(text))
#     except:
#         print("Sorry could not recognize what you said")

In [3]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

In [9]:
#give the file path to your audio file
audio_file_path = 'Recording.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.02 seconds.
(318, 256)


In [10]:
from pydub import AudioSegment

def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.split('.')[0] + '.wav'
    sound.export(audio_file_path, format="wav")
    return audio_file_path

audio_file_path = mp3_to_wav(audio_file_path)
print(audio_file_path)

Recording.wav


c:\users\swapn\appdata\local\programs\python\python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [11]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)

In [12]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [2]:
# text to speech module to convert the answer string into speech

# import pyttsx3
# engine = pyttsx3.init()
# ans = 'I live near my school'
# engine.say(ans)
# engine.runAndWait()